In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from datasets import load_dataset, Dataset, concatenate_datasets
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

In [2]:
model_name = "meta-llama/Llama-2-7b-hf"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.float16,
                                             device_map="auto"
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
from peft import PeftModel

model_name = "meta-llama/Llama-2-7b-hf"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             load_in_8bit=True,
                                             device_map="auto"
                                            )
tokenizer = AutoTokenizer.from_pretrained('enyuan/llama')
model.resize_token_embeddings(len(tokenizer))

model.load_adapter('enyuan/llama')

In [3]:
data = load_dataset("enyuan/Abstracts")
data_train = data["train"]

custom_data = load_dataset('json', data_files='data_eval.json')
data_val = custom_data['train']

# Print the dataset details
print(data_train)
print(data_val)

# Access an example
#example = data_train[0]
#print(example)

def generate_prompt(title, abstract=None, eos_token="</s>"):
  instruction = "The abstract of the paper:\n"
  input = f"{title}\n"
  abstract = f"Abstract: {abstract + ' ' + eos_token if abstract else ''} "
  prompt = (" ").join([instruction, input, abstract])
  return prompt

print(generate_prompt(data_train[0]["title"], data_train[0]["abstract"]))

Dataset({
    features: ['publicationDate', 'title', 'abstract', 'doi'],
    num_rows: 165071
})
Dataset({
    features: ['title', 'abstract', 'publicationDate'],
    num_rows: 559
})
The abstract of the paper:
 Inconel 625 sustainable milling surface integrity and the dependence on alloy processing route
 Abstract: The discovery of deepwater oil and gas sources has altered the scenario of world production of oil products, attracting even more attention to nickel superalloys. However, this class of materials can be used in several applications. Furthermore, nickel superalloys are highly dependent on their processing history, and the manner in which superalloys react to machining can directly affect the finished product. This work aims to evaluate the surface integrity of two different materials after cryogenic side-milling in conditions that stimulate severe plastic deformation (SPD) and high heat generation. The results show that the material response to machining depends strongly on 

In [6]:
input_prompt = generate_prompt(data_train[50]["title"])
input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
with torch.cuda.amp.autocast():
  generation_output = model.generate(
      input_ids=input_tokens,
      max_new_tokens=1000,
      do_sample=True,
      top_k=10,
      top_p=0.9,
      temperature=0.3,
      repetition_penalty=1.15,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
    )
op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
print(op)

The abstract of the paper:
 Effect of cryogenic cooling on residual stresses and surface finish of 316L during hybrid manufacturing
 Abstract:   In this work, the effect of cryogenic cooling (CC) on residual stress and surface roughness is investigated in a hybrid additive manufactured part. A 3D printed stainless steel (SS) component was subjected to CC treatment using liquid nitrogen (LN2). Residual stress measurements were carried out by X-ray diffraction method and surface roughness analysis was performed with white light interferometry technique. The results show that the residual stress profile decreases significantly after CC treatment. Surface roughness also reduces considerably due to the removal of oxide layer formed at the interface between LN2 and SS. This study demonstrates that CC can be used as an effective post processing step for improving mechanical properties and surface quality of AM parts.


In [4]:
with open('materials.txt', 'r') as file:
    word_list = file.read().splitlines()

In [6]:
new_data = {
    'title': word_list,
    'abstract': [s.replace('_', '') for s in word_list],
    'doi': ['material'] * len(word_list),  # 假设新数据集中没有doi信息
    'publicationDate': [None] * len(word_list)  # 假设新数据集中没有publicationDate信息
}
new_dataset = Dataset.from_dict(new_data)

data_train = concatenate_datasets([data_train, new_dataset])

new_data = {
    'title': [s.replace('_', '') for s in word_list],
    'abstract': word_list,
    'doi': ['material'] * len(word_list),  # 假设新数据集中没有doi信息
    'publicationDate': [None] * len(word_list)  # 假设新数据集中没有publicationDate信息
}
new_dataset = Dataset.from_dict(new_data)

data_train = concatenate_datasets([data_train, new_dataset])

In [7]:
data_val = data_train.select(range(200))

In [8]:
def generate_prompt(type, title, abstract=None, eos_token="</s>"):
    if type == 'material':
        instruction = "The material :\n"
        input = f"{title} is"
        output = f"{abstract + ' ' + eos_token if abstract else ''} "
        prompt = (" ").join([instruction, input, output])
    else:
        instruction = "The abstract of the paper:\n"
        input = f"{title}\n"
        output = f"Abstract: {abstract + ' ' + eos_token if abstract else ''} "
        prompt = (" ").join([instruction, input, output])
    return prompt

print(generate_prompt(data_train[-1]["doi"], data_train[-1]["title"], data_train[-1]["abstract"]))

The material :
 NiFeAlO4 is NiFeAlO_4 </s> 


In [9]:
lora_config = LoraConfig(
        r=128,
        lora_alpha=256,
        lora_dropout=0.05,
        target_modules=["q_proj","k_proj","v_proj","o_proj"],
        bias="none",
        task_type="CAUSAL_LM",
    )

In [10]:
# Step 1: Add new tokens to the tokenizer
num_added_toks = tokenizer.add_tokens(word_list)
print(f"Added {num_added_toks} tokens")

tokenizer.add_special_tokens({"pad_token": "<PAD>"})
model.resize_token_embeddings(len(tokenizer))

model = get_peft_model(model, lora_config)

Added 6110 tokens


In [11]:
# Step 2: Freeze all parameters in the model
#for param in model.parameters():
#    param.requires_grad = False

embeddings = model.get_input_embeddings()

# Enable gradient updates for the entire embedding layer
# Assuming you might want to fine-tune all embeddings, but here's how to selectively unfreeze
embeddings.weight.requires_grad = True

In [12]:
# 准备训练参数
training_args = TrainingArguments(
    output_dir='results',            # 输出目录
    num_train_epochs=1,              # 总训练轮数
    per_device_train_batch_size=4,   # 训练的batch size
    per_device_eval_batch_size=4,    # 验证的batch size
    gradient_accumulation_steps=4, 
    gradient_checkpointing=True,
    optim = "paged_adamw_32bit",
    bf16=True,
    #fp16=True,
    warmup_steps=10,                # 预热步数
    learning_rate = 1e-4,
    max_grad_norm = 0.2,
    #max_steps = 50,
    #warmup_ratio = 0.03,
    #weight_decay=0.01,               # 权重衰减
    save_strategy="steps",           # 设置保存策略为"steps"
    save_steps=300,                  # 每500步保存一次模型
    save_total_limit=3,              # 最多保存3个检查点
    evaluation_strategy="steps",     # 设置评估策略为"steps"
    group_by_length=True,
    eval_steps=5000
)

In [10]:
model.enable_input_require_grads()

In [13]:
def formatting_func(prompt):
  output = []

  for a, d, s in zip(prompt["doi"], prompt["title"], prompt["abstract"]):
    op = generate_prompt(a, d, s)
    output.append(op)

  return output


trainer = SFTTrainer(
    model=model,
    train_dataset=data_train,
    eval_dataset=data_val,
    peft_config=lora_config,
    formatting_func=formatting_func,
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_args
)

# We will also pre-process the model by upcasting the layer norms in float 32 for more stable training
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

trainer.train()
trainer.save_model(f"{output_dir}/final")

# Step Training Loss Validation Loss
# 10 1.848200 1.746341
# 20 1.688300 1.696681
# 30 1.654500 1.698127
# 40 1.579400 1.652010
# 50 1.492600 1.701877

Map:   0%|          | 0/189515 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB (GPU 0; 47.54 GiB total capacity; 45.97 GiB already allocated; 85.12 MiB free; 46.39 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [16]:
data_train

Dataset({
    features: ['abstract', 'doi', 'publicationDate', 'title'],
    num_rows: 189515
})

In [67]:
model.save_pretrained('result', save_embedding_layers=True)

In [68]:
tokenizer.save_pretrained('result')

('result/tokenizer_config.json',
 'result/special_tokens_map.json',
 'result/tokenizer.model',
 'result/added_tokens.json',
 'result/tokenizer.json')

In [34]:
tokenizer.save_pretrained('results')

('results/tokenizer_config.json',
 'results/special_tokens_map.json',
 'results/tokenizer.model',
 'results/added_tokens.json',
 'results/tokenizer.json')

In [ ]:
for param in model.parameters():
    print(param.dtype)

In [17]:
for name, param in model.named_parameters():
    print(param.requires_grad, name, param.dtype)

True base_model.model.model.embed_tokens.weight torch.float16
False base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight torch.float16
True base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight torch.float16
True base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight torch.float16
False base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight torch.float16
True base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight torch.float16
True base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight torch.float16
False base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight torch.float16
True base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight torch.float16
True base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight torch.float16
False base_model.model.model.layers.0.self_attn.o_proj.base_layer.weight torch.float16
True base_model.model.model.layers.0.self_attn.o_p

In [14]:
# Verify which parameters are trainable
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Trainable: {name}")

Trainable: base_model.model.model.embed_tokens.weight
Trainable: base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight
Trainable: base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight
Trainable: base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight
Trainable: base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight
Trainable: base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight
Trainable: base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight
Trainable: base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight
Trainable: base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight
Trainable: base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight
Trainable: base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight
Trainable: base_model.model.model.layers.1.self_attn.k_proj.lora_A.default.weight
Trainable: base_model.model.model.layers.1.s

In [ ]:
for name, param in model.named_parameters():
    print(f"Trainable: {name}", param.requires_grad)

In [14]:
model.parameters()

<generator object Module.parameters at 0x7faf7c779ee0>

In [15]:
model.get_input_embeddings()

Embedding(38544, 4096)

In [18]:
model.num_parameters()

6922694656

In [19]:
tokenizer.add_special_tokens({"pad_token": "<PAD>"})
model.resize_token_embeddings(len(tokenizer))

Embedding(38545, 4096)